In [16]:
# from gla_package.gla import aging_gompertz_makeham, learning_function, growth_function, fertility_brass_polynomial
from gla_package.gla import gla_model, gla_model_vectorized, aging_gompertz_makeham, learning_function, growth_function, fertility_brass_polynomial
# from gla_package.landscape import compute_fitness_grid_point
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import perf_counter
from joblib import Parallel, delayed
import scipy

In [17]:
import scipy.optimize

def survival_l(x, aging_function, learning_function, growth_function, aging_args, learning_args, growth_args):
	cumulative_hazard = scipy.integrate.quad(lambda t: gla_model(t, aging_function, learning_function, growth_function, aging_args, learning_args, growth_args), 0, x, limit=10000)[0]
	return np.exp(-cumulative_hazard)

def euler_lotka_function(r, aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args):
	integral = scipy.integrate.quad(lambda t: fertility_function(t, *fertility_args) * survival_l(t, aging_function, learning_function, growth_function, aging_args, learning_args, growth_args) * np.exp(-r*t), 0, 100, limit=1000)[0]
	return integral - 1

def compute_fitness(aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args):
    fitness = scipy.optimize.fsolve(lambda r: euler_lotka_function(r, aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args), 0.0001)[0]
    if fitness < 0:
        return 0.0
    return fitness

def compute_fitness_grid_point(grid_point, aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args):
    new_aging_args = [aging_args[0], grid_point[0], aging_args[2]]
    new_learning_args = [grid_point[1], learning_args[1], learning_args[2]]
    fitness = compute_fitness(aging_function, learning_function, growth_function, fertility_function, new_aging_args, new_learning_args, growth_args, fertility_args)
    return fitness 


In [19]:
def generate_grid(grid_type, b_min, b_max, lmax_min, lmax_max, dim = 5):
	grid = np.zeros((dim*dim, 2))

	if grid_type == "regular":
		b_grid = np.linspace(b_min, b_max, dim)
		Lmax_grid = np.linspace(lmax_min, lmax_max, dim)
		B, L = np.meshgrid(b_grid, Lmax_grid)

		grid[:, 0] = B.flatten()
		grid[:, 1] = L.flatten()
	return grid

def compute_reduced_b_grid(grid, b_decrease):
	reduced_b_grid = grid.copy()
	reduced_b_grid[:, 0] = reduced_b_grid[:, 0]*b_decrease
	return reduced_b_grid

def parallel_compute_fitness_grid(grid, aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args):
	fitness = Parallel(n_jobs=4)(delayed(compute_fitness_grid_point)(point, aging_function, learning_function, growth_function, fertility_function, aging_args, learning_args, growth_args, fertility_args) for point in grid)
	return fitness

	
aging_args = [0.00275961297460256,0.04326224872667336,0.025201676835511704]  # Arguments for aging_gompertz
learning_args = [0.01606792505529796,39.006865144958745,0.11060749334680318]  # Arguments for learning_function
growth_args = [0.05168141300917714,0.08765165352033985]  # Arguments for growth_function
fertility_args = [2.455e-5, 14.8, 32.836]  # Arguments for fertility_function


compute_fitness(aging_gompertz_makeham, learning_function, growth_function, fertility_brass_polynomial, aging_args, learning_args, growth_args, fertility_args)

0.03092579374637572